# LangChain QA Panel App

This notebook shows how to make this app:

In [ ]:
#!pip install langchain openai chromadb tiktoken pypdf panel


In [12]:
import os 
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import panel as pn
import tempfile


In [13]:
pn.extension('texteditor', template="bootstrap", sizing_mode='stretch_width')
pn.state.template.param.update(
    title="",
    main_max_width="690px",
    header_background="#92815b",
    logo="https://drive.google.com/uc?id=18exuDv-J4pZwzfN4rLU5Q9bGwDxV9BxI"
)

In [5]:
file_input = pn.widgets.FileInput(width=300)

# openaikey = pn.widgets.PasswordInput(
#     value="", placeholder="Enter your OpenAI API Key here...", width=300
# )
prompt = pn.widgets.TextEditor(
    value="", placeholder="Enter your questions here...", height=160, toolbar=False
)
run_button = pn.widgets.Button(name="Send Message")

select_k = pn.widgets.IntSlider(
    name="Number of relevant chunks", start=1, end=5, step=1, value=2
)
select_chain_type = pn.widgets.RadioButtonGroup(
    name='Chain type', 
    options=['stuff', 'map_reduce', "refine", "map_rerank"]
)

widgets = pn.Row(
    pn.Column(prompt, run_button, margin=5),
    pn.Card(
        "Chain type:",
        pn.Column(select_chain_type, select_k),
        title="Advanced settings", margin=10
    ), width=600
)

In [6]:
def qa(file, query, chain_type, k):
    # load document
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions 
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result

In [7]:
# result = qa("example.pdf", "what is the total number of AI publications?")

In [8]:
convos = []  # store all panel objects in a list

def qa_result(_):
    # API Key from Account
    os.environ["OPENAI_API_KEY"] = ""
    
    # save pdf file to a temp file 
    if file_input.value is not None:
        file_input.save("temp.pdf")
    
        prompt_text = prompt.value
        if prompt_text:
            result = qa(file="temp.pdf", query=prompt_text, chain_type=select_chain_type.value, k=select_k.value)
            convos.extend([
                pn.Row(
                    pn.panel("\U0001F60A", width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel("\U0001F916", width=10),
                    pn.Column(
                        result["result"])
                )
            ])
            #return convos
    return pn.Column(*convos, margin=15, width=575, min_height=400)


In [9]:
qa_interactive = pn.panel(
    pn.bind(qa_result, run_button),
    loading_indicator=True,
)

In [10]:
output = pn.WidgetBox('*Chat Box:*', qa_interactive, width=630, scroll=True)

In [11]:
# layout
pn.Column(
    pn.pane.Markdown("""
    ## ARIA Banking AI
    """),
    pn.Row(file_input),
    output,
    widgets

).servable()

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] FileInput(width=300)
        [1] PasswordInput(placeholder='Enter your OpenAI A...)
    [2] WidgetBox(scroll=True, width=630)
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] ParamFunction(function, _pane=Column, loading_indicator=True, sizing_mode='stretch_width')
    [3] Row(width=600)
        [0] Column(margin=5, sizing_mode='stretch_width')
            [0] TextEditor(height=160, placeholder='Enter your questions h..., sizing_mode='stretch_width', toolbar=False)
            [1] Button(name='Send Message', sizing_mode='stretch_width')
        [1] Card(margin=10, sizing_mode='stretch_width', title='Advanced settings')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Column(sizing_mode='stretch_width')
                [0] RadioButtonGroup(name='Chain type', options=['stuff', 'map_reduce', ...], sizing_mode='stretch_width', value='stuff')
                [1] IntSlider(end=5, name='Number of relevant c..., sizing_mode='stretch_width', start=1, value=2)